In [1]:
from nltk.corpus import stopwords
#talvez seja necessário baixar as stopwords com o comando no CMD: python -m nltk.downloader stopwords
from string import punctuation
from nltk.tokenize import TweetTokenizer
import re

#______________________________________________________________________________________________________________________#

#Instância um objeto do tipo TweetTokenizer que já deixa tudo em minúsculo, trata espaços,  
# e elimina menções (mas não tira números).
tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False) 

#definição das stopwords (inclui pontuação para eliminá-la, caso apareça)
stopwords = set(stopwords.words('portuguese') + list(punctuation)+ ['“','”','…','rt'])

def tokenizar_texto(texto_completo):
    texto_tokenizado_lst = tweet_tokenizer.tokenize(texto_completo)
    palavras_sem_stopwords = [palavra for palavra in texto_tokenizado_lst if palavra not in stopwords]
    return palavras_sem_stopwords

In [2]:
import nltk
import pandas as pd

# ler dados do arquivo seccionados a cada 200 tweets
dados = pd.read_json('tweets_222_23abr2020.json',convert_dates=False,lines=True,chunksize=2000)

In [3]:
# Criar Serie com os retweets dos 20000 primeiros tweets do arquivo
retweets = pd.Series(name = 'ID')

num = 0
for i in dados:
    num=num+1
    if(num<=50):
        retweets=retweets.append(i.dropna(subset=['retweeted_status'])['text'])
    else:
        break
retweets.size# 127121 retweets nos 200000 tweets # 12482 para 20000 # 62913 para 100000

C:\Users\vicda\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


62913

In [19]:
#criar dict para a ocorrencia e num_tweets dos Bigramas nos tweets
freq_bi = {}
num_tweets_bi = {}

#criar dict para a ocorrencia e num_tweets dos unigramas nos tweets
freq_uni = {}
num_tweets_uni = {}

add_uni = []# list para auxiliar na contagem de tweets em que aparece
add_bi = []# list para auxiliar na contagem de tweets em que aparece
for tweet in retweets.values:
    
    palavras = tokenizar_texto(tweet)
    bigramas_tweet = list(nltk.bigrams(palavras))
    add_uni.clear()
    add_bi.clear()
    
    for unigrama in palavras:
        if unigrama in freq_uni:
            freq_uni[unigrama] += 1
            if unigrama not in add_uni:
                num_tweets_uni[unigrama] += 1
                add_uni.append(unigrama)
        else:
            freq_uni[unigrama] = 1
            num_tweets_uni[unigrama] = 1
            add_uni.append(unigrama)
            
    for bigrama in bigramas_tweet:
        if bigrama in freq_bi:
            freq_bi[bigrama] += 1
            if bigrama not in add_bi:
                num_tweets_bi[bigrama] += 1
                add_bi.append(bigrama)
        else:
            freq_bi[bigrama] = 1
            num_tweets_bi[bigrama] = 1
            add_bi.append(bigrama)


In [30]:
# Criar Dataframe com os dict, associando as palavras ao index e ocorrencia e numero de tweeta presentes às colunas
Unigram = pd.DataFrame([freq_uni,num_tweets_uni]).T.rename(columns={0:'Ocorrências',1:'Tweets_Presente'})
Bigram = pd.DataFrame([freq_bi,num_tweets_bi]).T.rename(columns={0:'Ocorrências',1:'Tweets_Presente'})
Bigram.index.name = 'Bigramas'
Unigram.index.name = 'Unigramas'

In [31]:
Unigram.sort_values('Tweets_Presente',ascending=False).head()

,Ocorrências,Tweets_Presente
Unigramas,,
quarentena,26858,26280
covid,9649,9617
19,7290,7112
cloroquina,4578,4568
pra,4476,4296


In [32]:
Bigram.sort_values('Tweets_Presente',ascending=False).head()

,Ocorrências,Tweets_Presente
Bigramas,,
"(covid, 19)",7072,7045
"(quarentena, https://t.co/Bputjo9mS5)",3140,3140
"(nessa, quarentena)",1948,1948
"(testes, covid)",1525,1502
"(vai, matar)",1150,1150


In [33]:
Unigram.to_csv('Análise_Unigramas.csv',sep=';')
Bigram.to_csv('Análise_Bigramas.csv',sep=';')